In [42]:
#from IPython import get_ipython
#get_ipython().magic('reset -sf') 

In [43]:
import sys
from gensim.models import Word2Vec
from os import path
import numpy as np
import pandas as pd
pd.set_option('mode.chained_assignment', None)
from timeit import default_timer as timer
import itertools
import csv
import cplex
from math import inf as infinity
from scipy.spatial.distance import cdist
import bisect
from multiprocessing import Pool
import multiprocessing
from itertools import repeat
from itertools import islice, groupby

In [44]:
prefix                 = path.expanduser("/home/mcaserta/research/nlp/data")
ecco_models_folder     = "ecco_models/"
vocab_folder           = "google_vocab/"
modelnameW2V           = "word2vec.model.96-00.all"
premiumDocsXRowBase    = "/home/mcaserta/research/nlp/ecco_code/preproc/premiumDocsXRow.csv"
premiumCorpusXRowBase  = "/home/mcaserta/research/nlp/ecco_code/preproc/premiumCorpusXRow.csv"

In [45]:
def setupTarget(target, model):
    # setup target (invariant over cycle)
    nWords = len(target)
    demand = {key:0 for key in target}
    for w in target:
        demand[w] += 1
    nD     = len(demand)
    dem    = [val/nWords for val in demand.values()]
    D      = model.wv[demand.keys()]
    
    return nD, dem, D

def solveTransport2(matrixC, cap, dem, nS, nD):
    """
    Solve transportation problem as an LP.
    This is my implementation of the WMD.
    """
    
    cpx   = cplex.Cplex()
    x_ilo = []
    cpx.objective.set_sense(cpx.objective.sense.minimize)
    for i in range(nS):
        x_ilo.append([])
        for j in range(nD):
            x_ilo[i].append(cpx.variables.get_num())
            #  varName = "x." + str(i) + "." + str(j)
            cpx.variables.add(obj   = [float(matrixC[i][j])],
                              lb    = [0.0])
                              #  names = [varName])
    # capacity constraint
    for i in range(nS):
        index = [x_ilo[i][j] for j in range(nD)]
        #value = [1.0]*nD
        capacity_constraint = cplex.SparsePair(ind=index, val=[1.0]*nD)
        #capacity_constraint = cplex.SparsePair(ind=index, val=np.ones(nD))
        cpx.linear_constraints.add(lin_expr = [capacity_constraint],
                                   senses   = ["L"],
                                   rhs      = [cap[i]])

    # demand constraints
    for j in range(nD):
        index = [x_ilo[i][j] for i in range(nS)]
        #value = [1.0]*nS
        demand_constraint = cplex.SparsePair(ind=index, val=[1.0]*nS)
        cpx.linear_constraints.add(lin_expr = [demand_constraint],
                                   senses   = ["G"],
                                   rhs      = [dem[j]])
    cpx.parameters.simplex.display.set(0)
    cpx.solve()

    return cpx.solution.get_objective_value()


def wmdTransportNoLB(model, source, D, nD, dem):
        nWords   = len(source)
        capacity = {key:0 for key in source}
        #  print("SOURCE = ", source)
        for w in source:
            capacity[w] += 1

        nS       = len(capacity)
        cap      = [val/nWords for val in capacity.values()]
        try:
            S        = model.wv[capacity.keys()]
        except:
            return -1
        dd       = cdist(S,D)

        # solve transportation problem
        z = solveTransport2(dd, cap, dem, nS, nD)

        return z
    
def wmdTransport(model, sent, D, nD, dem, lastScore):
        source = sent.tokens
        nWords   = len(source)
        capacity = {key:0 for key in source}
        #print("SOURCE = ", source)
        for w in source:
            capacity[w] += 1

        nS       = len(capacity)
        cap      = [val/nWords for val in capacity.values()]
        try:
            S        = model.wv[capacity.keys()]
        except: #this might occur when the word is not in the dictionary
            return -1
        dd       = cdist(S,D)
        
        # compute lower bounds for fathoming
        lb = np.dot(dd.min(axis=1), cap)
        if lb >= lastScore:
            #sent.z=-1
            return sent
        
        lb = np.dot(dd.min(axis=0), dem)
        if lb > lastScore:
            #sent.z=-1
            return sent

        # if not pruned, solve transportation problem
        sent.z = solveTransport2(dd, cap, dem, nS, nD)
        
        return sent


In [46]:
def loadW2V():
    fullpath = path.join(prefix, ecco_models_folder)
    fullname = fullpath + modelnameW2V
    mW2V = Word2Vec.load(fullname)
    mW2V.init_sims(replace=True)
    return mW2V
#%time loadW2V()
global mW2V
mW2V = loadW2V()

In [47]:
#target=["sublime", "modification", "power"]
target=[ "power", "one"]
nD, dem, D = setupTarget(target, mW2V)

In [48]:
class Sentence:
    def __init__(self):
        self.tokens = []
        self.z      = -1
        self.id     = -1
#        self.year   = -1 # not used, verify
class Top():
    """
    Data structure used to store the top N sentences matching a given target
    sentence.
    We store both the full sentence (untokenized) and the tokenized and
    preprocessed sentence. We also store the previous and next sentences.
    """
    def __init__(self, nTop):
        self.score     = [-1]*nTop
        self.idnr      = [""]*nTop
        self.year      = [""]*nTop
        self.tokenSent = [[]]*nTop
        self.sent      = [""]*nTop
        self.prevSent  = [""]*nTop
        self.nextSent  = [""]*nTop
        self.idx       = [-1]*nTop
        self.best      = infinity
        self.star      = " "
        self.plus      = " "
            
    def getSortedList(self):
        return [self.score[i] for i in self.idx]

def populateFirstnTop(model, tops, year, nPopulate, readerDocs, D, nD, dem):
    # populate empty list
    forbiddenWords = ["one"]
    i = 0
    for source in islice(readerDocs, 0, nPopulate):
        found = False
        for w in source:
            if w in forbiddenWords:
                found = True
                break
        if found:
            continue
            
        #print("sentence for process ", multiprocessing.current_process().name, " = ", source)
        z = wmdTransportNoLB(mW2V, source, D, nD, dem)
        tops.score[i]     = z
        tops.tokenSent[i] = source        
        tops.idx[i]       = i
        tops.idnr[i]      = i
        tops.year[i]      = year
        i += 1
    # sort index w.r.t. score
    tops.idx = [x for _,x in sorted(zip(tops.score,tops.idx))]
    return tops
        
def sortedInsertion(tops, sent, year, nTop):
    #print("Insering", sent.z, "into", tops.getSortedList())
    #print("Index = ", tops.idx)
    last = tops.idx[-1]
    ss = [tops.score[i] for i in tops.idx]
    ss = tops.getSortedList()
    pos  = bisect.bisect(ss, sent.z)
    #print("in position ", pos)
    # add here information of the newly inserted item
    # (all the other fields do not need to be changed
    #  just move the idx value)
    tops.score[last]      = sent.z
    tops.idnr[last]       = sent.id
    tops.tokenSent[last]  = sent.tokens
    tops.year[last]       = year
    for i in range(nTop-2, pos-1, -1):
        tops.idx[i+1]       = tops.idx[i]
    tops.idx[pos] = last
    #print("Is sorted ? ", tops.getSortedList() == sorted(tops.score))
 
    return tops

def updateBest(tops, year, totSents, totPruned, i, totRead):
    if tops.score[tops.idx[0]] < tops.best:
        tops.best = tops.score[tops.idx[0]]
        tops.star = "*"
    else:
        tops.star = " "
    print("[{0:5.0f} secs. -{1}- {2:7d}/{3}] z* = {4:5.3f}\t [Fathomed : {5:7d} ({6:5.3f})] {7}{8}".format(timer()-start, year, i, totSents, tops.best, totPruned, totPruned/totRead, tops.plus, tops.star))
    tops.plus = " "
    return tops

def printTops(tops):
    for i,id in enumerate(tops.idx):
        print("[{0:4d}--{1}.{2:8d}] {3:5.3f} :: {4}".format(i, tops.year[id], tops.idnr[id], tops.score[id], tops.tokenSent[id]))

In [49]:
def wmdParallel(nCores, batch, printStep, years, tops, nTop, totPruned, totRead, totSentences):
    forbiddenWords = ["one"]
    p     = Pool(nCores)
    for year in years:
        premiumDocsXRow   = premiumDocsXRowBase + "." + year
        premiumCorpusXRow = premiumCorpusXRowBase + "." + year
        totSents = totSentences[year]
        totSents = 10000
        print("Loading sentences for year {0} [{1:7d} sentences]".format(year,totSents))

        with open(premiumDocsXRow, "r") as fDocs:
        #fDocs        = open(premiumDocsXRow, "r")
            readerDocs   = csv.reader(fDocs)
            nPopulate = 0
            if year == years[0]:
                nPopulate = min(nTop, totSents)
                tops = populateFirstnTop(mW2V, tops, year, nPopulate, readerDocs, D, nD, dem)
                tops = updateBest(tops, year, totSents, totPruned, nTop, nTop)

            batches = batch*nCores
            sources = []
            initIndex = nPopulate # to recover the index of each sentence

            for i,source in itertools.islice(enumerate(readerDocs), 0, totSents):
                found = False
                for w in source:
                    if w in forbiddenWords:
                        found = True
                        break
                if found:
                    continue                
                sent = Sentence()
                sent.tokens = source
                sent.id = nPopulate+i
                sources.append(sent)

                if i % printStep == 0:
                    totRead += printStep
                    tops = updateBest(tops, year, totSents, totPruned, i, totRead)

                if (i+1) % batches == 0 or i==totSents-1:
                    # divide and send
                    nSent = len(sources)
                    sets = np.array_split(np.arange(0,nSent,1), nCores)
                    slicedSources = [[ sources[s] for s in myset] for myset in sets]
                    results = p.starmap(getWMD,zip(slicedSources, repeat(tops.score[tops.idx[-1]]), repeat(D), repeat(nD), repeat(dem)) )

                    for cc in range(nCores):
                        for sent in results[cc]:
                            if sent.z == -1:
                                totPruned += 1
                            else:
                                if sent.z < tops.score[tops.idx[-1]]:
                                    tops.plus = "+"
                                    sortedInsertion(tops, sent, year, nTop)
                    sources = []
                    initIndex += nSent

            totRead += printStep
            tops = updateBest(tops, year, totSents, totPruned, i+1, totRead)
 
    p.close()       
        
    return tops, totPruned, totRead

def getWMD(sources, lastScore, D, nD, dem):
    return [wmdTransport(mW2V, sent, D, nD, dem, lastScore) for sent in sources]


In [50]:
def eccoWMD(nTop=10, batch=25, nCores=4, printStep=200000):
    
    global start
    global tops
    tops      = Top(nTop)

    years        = ["1796", "1797", "1798", "1799", "1800"]
    #years        = ["1796", "1797"]
    totSentences = {"1796":3280661, "1797":2945687,"1798":2857190, "1799":2622691, "1800":3098020 }

    totPruned = 0
    totRead   = 0

    start = timer()
    tops, totPruned, totRead = wmdParallel(nCores, batch, printStep, years, tops, nTop, totPruned, totRead, totSentences)

    print("Total Sentences Analyzed = ", totRead, "over a period of ", len(years), "years in ", timer()-start, " seconds.")

    return tops


In [51]:
def retrieveSentences(topsOut, nTop):
    # store it the "right way", no more pointers
    df = pd.DataFrame({
        'year'  : [topsOut.year[i] for i in topsOut.idx],
        'idnr'  : [topsOut.idnr[i] for i in topsOut.idx],
        'score' : [topsOut.score[i] for i in topsOut.idx],
        'tokenSent' : [topsOut.tokenSent[i] for i in topsOut.idx],
        'sent'      : ['']*nTop,
        'prevSent'  : ['']*nTop,
        'nextSent'  : ['']*nTop        
    })
    
    df = df.sort_values(['year','idnr'])
    #print("SORTED DF = ", df)
    groups = df.groupby('year')
    previousIndex = -1
    # check missing: if sentence is first or last of the file, i.e., idnr=0 or nSent
    for year, grouped_df in groups:
        premiumCorpusXRow = premiumCorpusXRowBase + "." + year
        with open(premiumCorpusXRow, "r") as fCorpus:
            readerCorpus   = csv.reader(fCorpus)

            currentPos = 1 # position of the header in file 
            # REM: after reading line e.g., 7, the header is positioned on line 8
            for index,el in grouped_df.iterrows():
                
                position    = el.idnr-currentPos 
                currentPos  = el.idnr + 3 # each time, we move three steps
                if position == -2:  # e.g., 10 and 11
                    df.prevSent[index] = df.sent[previousIndex]
                    df.sent[index]     = df.nextSent[previousIndex]
                    df.nextSent[index] = fCorpus.readline()
                elif position == -1: # e.g., 10 and 12
                    df.prevSent[index] = df.nextSent[previousIndex]
                    df.sent[index]     = fCorpus.readline()
                    df.nextSent[index] = fCorpus.readline()                    
                else: # e.g., 10 and 13
                    for line in islice(fCorpus, position, position+1, 1):
                        df.prevSent[index] = line
                    df.sent[index]     = fCorpus.readline()
                    df.nextSent[index] = fCorpus.readline()
                previousIndex = index 
    return df

In [52]:
#%lprun -f wmdTransport 
nTop = 10
tops = eccoWMD(nTop=nTop, batch=25, nCores=8)
df = retrieveSentences(tops, nTop)
df.to_csv("solution.csv")

Loading sentences for year 1796 [  10000 sentences]
[    0 secs. -1796-      10/10000] z* = 1.074	 [Fathomed :       0 (0.000)]  *
[    0 secs. -1796-       0/10000] z* = 1.074	 [Fathomed :       0 (0.000)]   
[    1 secs. -1796-   10000/10000] z* = 0.901	 [Fathomed :    8926 (0.022)] +*
Loading sentences for year 1797 [  10000 sentences]
[    1 secs. -1797-       0/10000] z* = 0.901	 [Fathomed :    8926 (0.015)]   
[    2 secs. -1797-   10000/10000] z* = 0.898	 [Fathomed :   18065 (0.023)] +*
Loading sentences for year 1798 [  10000 sentences]
[    2 secs. -1798-       0/10000] z* = 0.898	 [Fathomed :   18065 (0.018)]   
[    2 secs. -1798-   10000/10000] z* = 0.898	 [Fathomed :   27259 (0.023)] + 
Loading sentences for year 1799 [  10000 sentences]
[    2 secs. -1799-       0/10000] z* = 0.898	 [Fathomed :   27259 (0.019)]   
[    3 secs. -1799-   10000/10000] z* = 0.877	 [Fathomed :   36517 (0.023)] +*
Loading sentences for year 1800 [  10000 sentences]
[    3 secs. -1800-       0/1

In [56]:

df


,idnr,nextSent,prevSent,score,sent,tokenSent,year
5,3077,"""I Continually haunting the offices of the war...","""PART I. H time ( 98) time and attention were ...",0.922747,"""ment, fortune, and power.""\n","[ment, fortune, power]",1796
3,9136,"""The minds of the people are already angered w...","""It is in vain to dissemble that this country ...",0.900644,Mr. Pitt has it in his power to save himself a...,"[power, save, empire]",1796
2,248,"""By Power, is meant 4 Almighty abilit 6. to co...","""41. minion of both, because &#x00B0; Prayer i...",0.898427,"""q all Dominion, Power, and Glory.""\n","[dominion, power, glory]",1797
4,1549,The coefficient of each term is obtained by mu...,"""On the contrary, b is wanting in the first te...",0.912033,"""In all the terms, the sum of the exponents of...","[terms, sum, equal, power]",1797
6,4570,"""Gaolers or printer of the Gazette not complyi...","""with treble colts; the debtor&#x0027; s right...",0.933549,"""Power of leasing lands yelled in assignee,.""\n","[power, lands, assignee]",1798
7,7862,"""alone belongs all the higher pursuits, of kno...","""indeed everything is guided by these, a few f...",0.955693,"""belongs all; power and authority, public and ...","[belongs, power, authority, public, private]",1798
9,9495,"""Wre would be glad to see the persons condesce...","""How came their frame to be different from, na...",0.962934,"""If so, why have they more impediments, and le...","[impediments, less, power, obedience]",1798
1,2988,"""IT is a singular circumstance in the history ...","""The house of peers have an officer called a c...",0.883100,Its legislative Power aidt Privileges.\n,"[legislative, power, privileges]",1799
0,4583,But you know that ours are not kings by eletio...,"""What racks must the man, who has these exampl...",0.877173,"""For nations, who have the power of eleeting k...","[nations, power, kings, also, power, binding]",1799
8,4115,"""And such powers and conditions may be execute...","""Nor to estates to her on condition to sell, i...",0.959218,"""Or with a power annexed to them, ibid.""\n","[power, annexed, ibid]",1800


In [54]:
#%load_ext line_profiler
#%lprun -f retrieveSentences retrieveSentences(tops,nTop)

In [55]:
#!python -m line_profiler script_to_profile.py.lprof